# Prerequisites

In [1]:
!git clone git@github.com:microsoft/promptbench.git

Cloning into 'promptbench'...
remote: Enumerating objects: 1457, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 1457 (delta 282), reused 242 (delta 242), pack-reused 1125 (from 2)
Receiving objects: 100% (1457/1457), 5.71 MiB | 7.43 MiB/s, done.
Resolving deltas: 100% (835/835), done.


In [ ]:
!pip install -r promptbench/requirements.txt


In [ ]:

!pip install -U auto-gptq

In [2]:
import sys

sys.path.append('./promptbench')
import promptbench as pb

/home/rafadc/code/tfm_2025/promptbench/promptbench/utils/dataprocess.py:62: SyntaxWarning: invalid escape sequence '\{'
  pred = pred.lower().replace("<pad>", "").replace("</s>", "").strip(",._\"\'-+=!?()&^%$#@:\\|\{\}[]<>/`\n\t\r\v\f ")


ModuleNotFoundError: No module named 'autocorrect'

# Login

Depending on the model you may need to login. At the moment this is not needed

In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()

# Load dataset

In [ ]:
from datasets import get_dataset_config_names

configs = get_dataset_config_names("hendrycks_test", trust_remote_code=True)
print(configs)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu", "abstract_algebra", trust_remote_code=True, download_mode="force_redownload")  # or "all"
test_set = dataset["test"]

print(dataset["test"][0])
print(dataset["test"][1])
print(dataset["test"][5])
print(dataset["test"][6])


# Generate prompt templates

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cpu"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-7B-Chat",
    torch_dtype="auto",
    device_map={"": device}
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-7B-Chat")

# Example question from your dataset
original_question = dataset["test"][0]["question"]

# Prompt template
prompt = (
    "Give me a prompt to calculate the sentiment in an LLM.\n"
    "Use {content} where you want to put the sentence you are analyzing"
    "Examples:\n"
    "Label the sentiment expressed in the sentence as positive or negative: {content}"
    "Determine if the given sentence expresses a positive or negative sentiment: {content}"
    "Conclude if the emotional tone of this sentence is positive or negative: {content}"
    "Recognize whether the sentiment of the following statement is positive or negative: {content}"
)

# Chat template
messages = [
    {"role": "system", "content": "You are my assistant in rewriting questions for an exam so the students can understand better what is being asked."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Tokenize once and reuse inputs
model_inputs = tokenizer([text], return_tensors="pt").to(device)

# Generate multiple variants
variants = []
for _ in range(50):
    generated_ids = model.generate(
        model_inputs.input_ids,
        attention_mask=model_inputs.attention_mask,
        max_new_tokens=128,
        do_sample=True,          # Enable sampling
        temperature=0.9,         # Control randomness
        top_p=0.95,              # Nucleus sampling
        top_k=50,                # Optional: top-k sampling
        repetition_penalty=1.1   # Slight penalty to avoid near-duplicates
    )

    # Remove the prompt portion from the output
    new_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
    response = tokenizer.decode(new_ids, skip_special_tokens=True).strip()
    variants.append(response)

# Done — all 50 responses are now in `variants`
print(variants[:3])  # Show a few examples



In [ ]:
print(prompt)

In [ ]:
for i, variation in enumerate(variants):
    print(f"Variation {i+1}:\n{variation}\n")



# Evaluate the prompt template using PromptEval

In [ ]:
import promptbench as pb

# print all supported datasets in promptbench
print('All supported datasets: ')
print(pb.SUPPORTED_DATASETS)

# load a dataset, sst2, for instance.
# if the dataset is not available locally, it will be downloaded automatically.
dataset = pb.DatasetLoader.load_dataset("sst2")
# dataset = pb.DatasetLoader.load_dataset("mmlu")
# dataset = pb.DatasetLoader.load_dataset("un_multi")
# dataset = pb.DatasetLoader.load_dataset("iwslt2017", ["ar-en", "de-en", "en-ar"])
# dataset = pb.DatasetLoader.load_dataset("math", "algebra__linear_1d")
# dataset = pb.DatasetLoader.load_dataset("bool_logic")
# dataset = pb.DatasetLoader.load_dataset("valid_parenthesesss")

# print the first 5 examples
dataset[:5]

In [ ]:
# print all supported models in promptbench
print('All supported models: ')
print(pb.SUPPORTED_MODELS)

# load a model, flan-t5-large, for instance.
# model = pb.LLMModel(model='google/flan-t5-large', max_new_tokens=10, temperature=0.0001, device='cuda')
model = pb.LLMModel(model='llama2-13b-chat', max_new_tokens=10, temperature=0.0001)

In [ ]:
# using different prompts to evaluate models
prompt_list = [
    "Classify the sentence as positive or negative: {content}",
    "Determine the emotion of the following sentence as positive or negative: {content}",
    "Is the sentiment of this sentence positive or negative? {content}",
    "Identify whether the sentiment in the following sentence is positive or negative: {content}",
    "Assess the sentiment of this statement as either positive or negative: {content}",
    "Evaluate the following sentence and indicate if it is positive or negative: {content}",
    "Judge the emotional tone of this sentence as positive or negative: {content}",
    "Label the sentiment expressed in the sentence as positive or negative: {content}",
    "Decide if the sentiment in this statement is positive or negative: {content}",
    "Analyze the following sentence and determine if it is positive or negative: {content}",
    "Categorize the sentiment of the given sentence as positive or negative: {content}",
    "Tell if the following sentence conveys a positive or negative sentiment: {content}",
    "Discern whether the emotion in the sentence is positive or negative: {content}",
    "Determine if the given sentence expresses a positive or negative sentiment: {content}",
    "Conclude if the emotional tone of this sentence is positive or negative: {content}",
    "Recognize whether the sentiment of the following statement is positive or negative: {content}",
    "Rate the sentiment in this sentence as positive or negative: {content}",
    "Classify the emotional tone of the given sentence as positive or negative: {content}",
    "Identify the sentiment in this sentence and classify it as positive or negative: {content}",
    "Assess if the sentiment of the following statement is positive or negative: {content}",
    "Indicate whether the sentiment of this sentence is positive or negative: {content}",
    "Determine if the sentiment in this sentence is positive or negative: {content}",
    "Judge whether the following sentence has a positive or negative sentiment: {content}",
    "Analyze the emotional tone of the sentence and classify it as positive or negative: {content}",
    "Label the given sentence as having a positive or negative sentiment: {content}",
    "Evaluate whether the sentiment in the following sentence is positive or negative: {content}",
    "Categorize the given sentence based on whether its sentiment is positive or negative: {content}",
    "Determine the emotional quality of the sentence as positive or negative: {content}",
    "Is the emotional tone of this sentence positive or negative? {content}",
    "Discern the sentiment of the given sentence and label it as positive or negative: {content}"
]

In [ ]:
def proj_func(pred):
    mapping = {
        "positive": 1,
        "negative": 0
    }
    return mapping.get(pred, -1)

In [ ]:
from promptbench.prompteval import efficient_eval

result = efficient_eval(model, prompt_list, dataset, proj_func, 
                        budget=1200,  # The maximum number of examples that can be evaluated during the process. Increasing this value covers more data points, while decreasing it reduces computation.
                        visualize=True,  # If set to True, the function will generate and display visualizations of the model's performance (combined_result.png), including histograms, boxplots, and cumulative distribution functions (CDFs).
                        pca_dim=25,  # The number of dimensions retained during PCA on the prompt embeddings. Higher values retain more dimensional information, while lower values reduce dimensionality.
                        method='EmbPT')  # The evaluation method to use. 'EmbPT' involves embedding the prompts and using these embeddings in model fitting. 'Rasch' does not obtain prompt embeddings; instead, prompts are one-hot encoded in this method.

print(result)

In [ ]:
def get_max_performance_index(data):
    """
    Returns the index of the highest value in the 'full_performances' list.

    Parameters:
        data (dict): Dictionary containing a 'full_performances' key with a list of floats.

    Returns:
        int: Index of the maximum performance value.
    """
    return int(np.argmax(data['full_performances']))


In [ ]:
import numpy as np


index = get_max_performance_index(result)
print("Index of highest full_performance:", index)
print(f"Best prompt template is: {prompt_list[index]}")

# Model evaluation

We have to evaluate again MMLU after using the best template we found on a given dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import bitsandbytes as bnb

model_id = "TheBloke/deepseek-llm-7B-chat-GGUF"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model using 8-bit quantization with bnb.nn.Linear8bitLt
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto",
)

model.eval()

In [ ]:
def format_prompt(example):
    choices = example["choices"]
    return (
        f"{example['question']}\n"
        f"A. {choices[0]}\n"
        f"B. {choices[1]}\n"
        f"C. {choices[2]}\n"
        f"D. {choices[3]}\n"
        f"Answer:"
    )

In [ ]:
def ask_model(prompt, max_new_tokens=5):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = decoded[len(prompt):].strip().upper()
    print(f"Prompt: {prompt}")
    print(f"Answer: {answer}")
    for option in ['A', 'B', 'C', 'D']:
        if answer.startswith(option):
            return option
    return "Invalid"

In [ ]:
import openai

client = openai.OpenAI(
    api_key=os.getenv("OPEN_AI_API_KEY")
)

def ask_gpt4(prompt, max_tokens=5):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that is answering test questions. Answer with just one letter. For example: B."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=max_tokens,
            temperature=1.0,
        )
        answer = response.choices[0].message.content.strip().upper()
        print(f"Prompt: {prompt}")
        print(f"Answer: {answer}")
        for option in ['A', 'B', 'C', 'D']:
            if answer.startswith(option):
                return option
        return "Invalid"
    except openai.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return "Error"


In [ ]:
print(test_set[0].keys())


In [ ]:
from tqdm import tqdm

letter_to_index = {"A": 0, "B": 1, "C": 2, "D": 3}
index_to_letter = {0: "A", 1: "B", 2: "C", 3: "D"}

correct = 0
total = 0

for example in tqdm(test_set.select(range(10))):  # Use full set if needed
    prompt = format_prompt(example)
    pred_letter = ask_model(prompt)
    #pred_letter = ask_gpt4(prompt)
    pred_index = letter_to_index.get(pred_letter, -1)  # -1 if invalid
    real_answer_letter = index_to_letter[example["answer"]]

    print(f"Real answer: {real_answer_letter}")
    print(f"Predicted answer: {pred_letter}")

    if pred_letter == real_answer_letter:
        correct += 1
    total += 1

print(f"Accuracy: {correct / total:.2%}")